 Selection du modèle

 Dans notre projet, nous traitons un problème de classification binaire où nous devons classer le compte Instagram d'un utilisateur donné comme réel ou faux. Par conséquent, nous nous limiterons à expliquer les différents algorithmes de classification binaire possibles et nous comprendrons pourquoi ils sont considérés comme adaptés à notre projet dans les sections ultérieures.

Dans un premier temps nous avons importer les librairies nécessaires pour l'implémentation des différents modèles.
Ensuite, nous avons importer et diviser le jeu de données en données d'entrainement et données de test

In [1]:
import random
import json
import pandas as pd
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout

from numpy.random import seed



# file to use differents ML models and compare them

# load  the data 
#users_json = json.load(open('../data/accountData_clean.json'))

data = pd.read_json('../data/accountData_clean.json')
    
y = data.isFake
X = data.drop(['isFake'], axis=1)

#split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=242)


Support Vector Machine (SVM) : 
le SVM est un classificateur à marge maximale. Il s'appuie sur les vecteurs de support pour définir une limite de décision sur un ensemble de données.

In [2]:
def svm_Training ():
       
    #Import svm model 
    from sklearn import svm

    #create a svm classifier
    clf = svm.SVC(kernel='linear', C=100.0) # Linear Kernel

    #train the model
    clf.fit(X_train,y_train)

    #predict the response for test dataset
    y_pred = clf.predict(X_test)

    #Import scikit-learn metrics module for accuracy calculation
    from sklearn import metrics

    # Model Accuracy: how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


    # Model confusion   
    from sklearn.metrics import confusion_matrix        
    print(confusion_matrix(y_test, y_pred))

    # Model Precision
    #from sklearn.metrics import precision_score
    #print("Precision:",precision_score(y_test, y_pred))

svm_Training()

Accuracy: 0.9937238493723849
[[387   1]
 [  2  88]]


Neural network :
Les réseaux neuronaux sont des simulations artificielles du réseau neuronal biologique du cerveau humain dans des ordinateurs. Ils utilisent des perceptrons, un équivalent artificiel des neurones biologiques. Les neurones sont connectés les uns aux autres de manière hiérarchique.

In [3]:
#data preprocessing for NN

X_train = data[pd.notnull(data['isFake'])].drop(['isFake'],axis=1)
y_train= data[pd.notnull(data['isFake'])]['isFake']
X_test = data[pd.isnull(data['isFake'])].drop(['isFake'],axis=1)

def NN_Training(lyrs=[8], act='linear', opt='Adam', dr=0.0):


    #data preprocessing
   

      # set random seed for reproducibility
    seed(42)
    tensorflow.random.set_seed(42)
    
    model = Sequential()
    
    # create first hidden layer
    model.add(Dense(lyrs[0], input_dim=X_train.shape[1], activation=act))
    
    # create additional hidden layers
    for i in range(1,len(lyrs)):
        model.add(Dense(lyrs[i], activation=act))
    
    # add dropout, default is none
    model.add(Dropout(dr))
    
    # create output layer
    model.add(Dense(1, activation='sigmoid'))  # output layer
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    print(model.summary())

    # fit the model
    training = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    #print(training.history)
    val_acc = np.mean(training.history['val_accuracy'])
    print("\n%s: %.2f%%" % ('val_accuracycl', val_acc*100))

    # evaluate the model
    scores = model.evaluate(X_train, y_train)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    # make  predictions on test set
    #test['isFake'] = model.predict(X_test)
    #test['isFake'] = test['isFake'].apply(lambda x: round(x,0)).a
    #test['isFake'] = test['isFake'].astype(int)

    #solution(test[['id','isFake']])

NN_Training()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 80        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________
None

val_accuracycl: 97.03%
38/38 [==============================] - 0s 3ms/step - loss: 0.1425 - accuracy: 0.9707

accuracy: 97.07%
